In [1]:
%matplotlib notebook  

In [2]:
!nvidia-smi

Tue Jul 27 15:36:37 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.56       CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  GeForce RTX 208...  On   | 00000000:18:00.0 Off |                  N/A |
| 53%   84C    P2   215W / 250W |   5133MiB / 11019MiB |     95%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  GeForce RTX 208...  On   | 00000000:3B:00.0 Off |                  N/A |
| 59%   

In [3]:
import matplotlib.pyplot as plt
import numpy as np
import time
import torch
import pandas as pd

# Python 3 standard library
from pathlib import Path

from model.efficiency import pv_locations, efficiency

##from torchsummary import summary

In [4]:
# From model/collectdata.py
from model.collectdata_kde_Ellipsoids import collect_t2kde_data
from model.collectdata_mdsA  import collect_truth

# From model/loss.py
##from loss import Loss
##from model.kde_loss_D import Loss
# from model.kde_loss_B import Loss


##from model.models_kde import TracksToKDE_Ellipsoids_DirtyDozen as Model
from model.models_kde import TracksToKDE_Ellipsoids_DDplus as Model

from model.training_kde import trainNet, select_gpu, Results
from model.plots import dual_train_plots, replace_in_ax

In [5]:
results = pd.DataFrame([], columns=Results._fields)

In [6]:
#device = select_gpu(0)
device = "cpu"

In [7]:
# Validation dataset. You can slice to reduce the size.
## mds no separate validation set yet,
batch_size = 16
val_loader = collect_t2kde_data('dataAA/20K_POCA_kernel_evts_200926.h5',
                                batch_size=batch_size,
                                device=device,
                                slice = slice(0,20))

PV = collect_truth('dataAA/20K_POCA_kernel_evts_200926.h5', pvs=True)
print('PV.n.shape =    ',  PV.n.shape)
print('PV.n[0].shape = ', *PV.n[0].shape)
print('PV.x[0] =       ', *PV.x[0])
print('PV.y[0] =       ', *PV.y[0])
print('PV.z[0] =       ', *PV.z[0])
print('PV.n[0] =       ', *PV.n[0])
print('PV.cat[0] =     ', *PV.cat[0])

SV = collect_truth('dataAA/20K_POCA_kernel_evts_200926.h5', pvs=False)
print('SV.n.shape =    ', SV.n.shape)
print('SV.n[0].shape = ', *SV.n[0].shape)
print('SV.x[0] =       ', *SV.x[0])
print('SV.y[0] =       ', *SV.y[0])
print('SV.z[0] =       ', *SV.z[0])
print('SV.n[0] =       ', *SV.n[0])
print('SV.cat[0] =     ', *SV.cat[0])

Loading data...
pocaMx.shape =  (20000,)
nEvts =  20000
len(pocaMx[0]) =  211
len(pocaMx[1]) =  21
len(pocaMx[2]) =  20
len(pocaMx[3]) =  198
len(pocaMx[4]) =  233
majorAxis.shape =  (20000, 3)
minorAxis_1.shape =  (20000, 3)
minorAxis_2.shape =  (20000, 3)
have entered six_ellipsoid_parameters
  
 
  nEvts =  20000
 iEvt, nTrks =  0 211
 iEvt, nTrks =  1 21
 iEvt, nTrks =  2 20
 iEvt, nTrks =  3 198
 iEvt, nTrks =  4 233
 iEvt, nTrks =  5 85
 iEvt, nTrks =  6 223
 iEvt, nTrks =  7 425
 iEvt, nTrks =  8 252
 iEvt, nTrks =  9 169
A.shape =  (20000,)
majorAxis[iTrk][0][0] =  -0.00023452607
majorAxis[iTrk][1][0] =  -0.00047206535
majorAxis[iTrk][2][0] =  0.096502915
minorAxis_1[iTrk][0][0] =  -15.822749
minorAxis_1[iTrk][1][0] =  7.8608756
minorAxis_1[iTrk][2][0] =  -2.6228399e-08
minorAxis_2[iTrk][0][0] =  7.860759
minorAxis_2[iTrk][1][0] =  15.822513
minorAxis_2[iTrk][2][0] =  0.096502915
  
majorAxis[iTrk][0][0] =  -0.37655562
majorAxis[iTrk][1][0] =  0.2768704
majorAxis[iTrk][2][0] = 

In [8]:
architectures = [[50]*11]
#model_names = []
latentChannels = 4
models = {}
models_dict = {}

nOut1,nOut2,nOut3,nOut4,nOut5,nOut6,nOut7,nOut8,nOut9,nOut10,nOut11 = architectures[0]

for i in range(50):
    models[i] = Model(nOut1,nOut2,nOut3,nOut4,nOut5,nOut6,nOut7,nOut8,nOut9,nOut10,nOut11,latentChannels)
    models_dict[i] = models[i].state_dict()
#model_latest = Model(nOut1,nOut2,nOut3,nOut4,nOut5,nOut6,nOut7,nOut8,nOut9,nOut10,nOut11,latentChannels)
#model_old = Model(nOut1,nOut2,nOut3,nOut4,nOut5,nOut6,nOut7,nOut8,nOut9,nOut10,nOut11,latentChannels)

In [9]:
folders = {}
for i in range(50):
    folders[i] = ''
    
# folders[1]  = '22December_testing_DDplus_loss_Bb_iter1_floatAll_100epochs_1em5'
# #folders[4]  = '25December__DDplus_loss_Ba_iter4_floatAll_500epochs_1em5'
# folders[7]  = '25December__DDplus_loss_Ba_iter7_floatAll_800epochs_4em6'
# folders[27] = '05Feb_DDplus_loss_Ba_iter27_floatAll_100epochs_1p5em5_4xwill'
# folders[28] = '06Feb_DDplus_loss_Ba_iter28_floatAll_100epochs_1p5em5_4xwill'
# iters = [1,7,27,28]

folders[1]  = '23July2021_DDplus_loss_Ba_iter29_floatAll_SetVar_20_nodes_it29pretrain_250_epochs_1e-05'
#folders[4]  = '25December__DDplus_loss_Ba_iter4_floatAll_500epochs_1em5'
folders[7]  = '23July2021_DDplus_loss_Ba_iter29_floatAll_SetVar_50_nodes_it29pretrain_250_epochs_1e-05'
folders[27] = '23July2021_DDplus_loss_Ba_iter29_floatAll_SetVar_5to100_nodes_it29pretrain_250_epochs_1e-05'
folders[28] = '23July2021_DDplus_loss_Ba_iter29_floatAll_SetVar_100_nodes_it29pretrain_250_epochs_1e-05'
iters = [1,7,27,28]


mycolors = {1:'green',7:'darkorange',27:'red',28:'magenta'}

In [10]:
suffix = 'final'

weights_dicts = {}
pretrained_dicts = {}
for i in folders.keys():
    if not i in iters:
        continue
    rootDir = 'jgo_files/'
    if i>18:
        rootDir = 'jgo_files/'        
    weights_dicts[i] = rootDir + folders[i] + '/' + folders[i] + '_'+ suffix + '.pyt'
    print(weights_dicts[i])
    pretrained_dicts[i] = torch.load(weights_dicts[i])
    # 1. filter out unnecessary keys
    pretrained_dicts[i] = {k: v for k, v in pretrained_dicts[i].items() if k in models_dict[i]}
    # 2. overwrite entries in the existing state dict
    models_dict[i].update(pretrained_dicts[i]) 
    # 3. load the new state dict
    models[i].load_state_dict(pretrained_dicts[i],strict=False)

jgo_files/23July2021_DDplus_loss_Ba_iter29_floatAll_SetVar_20_nodes_it29pretrain_250_epochs_1e-05/23July2021_DDplus_loss_Ba_iter29_floatAll_SetVar_20_nodes_it29pretrain_250_epochs_1e-05_final.pyt


RuntimeError: Error(s) in loading state_dict for TracksToKDE_Ellipsoids_DDplus:
	size mismatch for layer1.weight: copying a param with shape torch.Size([20, 9]) from checkpoint, the shape in current model is torch.Size([50, 9]).
	size mismatch for layer1.bias: copying a param with shape torch.Size([20]) from checkpoint, the shape in current model is torch.Size([50]).
	size mismatch for layer2.weight: copying a param with shape torch.Size([20, 20]) from checkpoint, the shape in current model is torch.Size([50, 50]).
	size mismatch for layer2.bias: copying a param with shape torch.Size([20]) from checkpoint, the shape in current model is torch.Size([50]).
	size mismatch for layer3.weight: copying a param with shape torch.Size([20, 20]) from checkpoint, the shape in current model is torch.Size([50, 50]).
	size mismatch for layer3.bias: copying a param with shape torch.Size([20]) from checkpoint, the shape in current model is torch.Size([50]).
	size mismatch for layer4.weight: copying a pa

In [ ]:
fig_size = plt.rcParams["figure.figsize"]
fig_size[0] = 10
fig_size[1] = 4
plt.rcParams["figure.figsize"] = fig_size


# output_folder = 'jgo_files/Tracks_to_KDE_TestTrainedModel/Model_DDplus_CompIters'
# for it in iters:
#     output_folder += '_%s'%(it)
# output_path = Path(output_folder)
# output_path.mkdir(exist_ok=True)


In [ ]:
for i in models.keys():
    models[i].to(device)

In [ ]:
# dict to store the outputs of the trained models (iteration over same model)
outputs = {}
# dict to store the predicted KDEs from the trained models (iteration over same model)
y_pred = {}

In [ ]:
nFeatures = 4000
nTracks_good_PV = 5
maxEvt = 5
plt_name = ''
#for it in iters:
#    plt_name+='%s_'%(it)

with torch.no_grad():
    print("val_loader = ",val_loader)
    # Loop over the trained iterations to retrieve the predictions 
    for it in iters:
        outputs[it] = models[it](val_loader.dataset.tensors[0])

    # Get the number of events from the output of the first trained model 
    nEvts = outputs[iters[0]].shape[0]
    #nEvts = 5
    
    print("nEvts = ",nEvts)
    labels = val_loader.dataset.tensors[1]

    # Do something on the true distribution (KDEs) 
    y = labels.view(nEvts,-1,nFeatures)
    y = y.transpose(1,2)               
    #print("y.shape = ",y.shape)
    
    # LOOP over the events
    for iEvt in range(maxEvt):

        print("iEvt = ",iEvt)
        
        # --- Retrieve the TRUE KDEs
        y_kde = y[iEvt,:,0].cpu().numpy()
        # print("y_kde.shape = ",y_kde.shape)

        # --- Retrieve the PREDICTED KDEs for each trained model (iteration over same model)
        for it in iters:
            y_pred[it] = outputs[it][iEvt,:]
            y_pred[it] = y_pred[it].cpu().numpy()
        
        pv_sorted = sorted(zip(PV.z[iEvt],PV.cat[iEvt],PV.n[iEvt]), key=lambda x: x[0])
        good_pvs = []
        good_pv_bins = []
        for ii in range(len(pv_sorted)):
            
            # PV with at least 5 tracks are considered as good PVs
            # Can also filter using "pv_sorted[ii][1]", which is a boolean at 1 if nTracks_good_PV is >= 5
            if pv_sorted[ii][2]>=nTracks_good_PV:
                good_pvs.append(pv_sorted[ii])
                good_pv_bins.append(np.floor(10.*(pv_sorted[ii][0]+100)))
        #print("good_pvs = ",good_pvs)
        #print("good_pv_bins = ", good_pv_bins)
        n_good_pv_bins = len(good_pv_bins)
        
        pvstr = "PVs" # tag for the text box info on the plot
        if len(good_pv_bins)<=1:
            pvstr = "PV"
      
        # --- Plot things here
        fig, ax = plt.subplots()
        plt.ylim(0.0005,1.15*max(y_kde))
        # --- Add vertical line showing the good PVs position
        for ipv in range(n_good_pv_bins):
            plt.axvline(x=good_pv_bins[ipv], color='grey', linestyle='dashed')
        plt.axvline(x=good_pv_bins[ipv], color='grey', linestyle='dashed', label='Good %s'%(pvstr))
        # --- Add the true KDE distribution
        plt.plot(y_kde, color="royalblue", label="True KDE")
        # --- Add the predicted KDE distributions (for each model iteration)
        for it in iters:
            plt.plot(y_pred[it], color=mycolors[it], alpha = 0.8, label="Pred. KDE (iter %s)"%(it))
        plt.legend(loc=1, facecolor='white', edgecolor='white')
        

        # --- Text box info added on the top left part of the plot, with
        # - event number
        # - number of good PVs out of the total number of PVs for the event


        textstr = '\n'.join((
        r'Evt #%s' % (iEvt, ),     
        #r'$-$ %s good %s (of %s)' %(len(good_pv_bins),pvstr,len(pv_sorted)) 
        ))              
        props = dict(boxstyle='square', facecolor='white', edgecolor='white', alpha=0.5) #
        ax.text(0.05, 0.95, textstr, transform=ax.transAxes,verticalalignment='top', bbox=props,linespacing = 1.8)#, fontsize=14
        ax.set_xlabel("z (in bin #)")
        ax.set_ylabel("Arbitrary unit")
        fname = 'Evt%s'%(iEvt)
        #plt.savefig(str(output_path / f'{fname}.png'))

   

        # Make the zoomed in plots for each of the good PV
        for jj in range(n_good_pv_bins):
            print("PV position and bin # = ", good_pvs[jj], good_pv_bins[jj])
            
            nLow =  int(good_pv_bins[jj]-50)
            nHigh = int(good_pv_bins[jj]+50)
            x=[i+nLow for i in range(100)]

            fig, ax = plt.subplots()
            #plt.figure()
            plt.xlim(x[0],x[99])
            plt.ylim(0.0005,1.25*max(y_kde[nLow:nHigh]))
            # --- Add vertical line showing the good PV position
            plt.axvline(x=good_pv_bins[jj], color='black', linestyle='dashed', label="True PV position")
            # --- Add the true KDE distribution
            plt.plot(x,y_kde[nLow:nHigh], color="royalblue", label="True  KDE")
            # --- Add the predicted KDE distributions (for each model iteration)
            for it in iters:
                plt.plot(x,y_pred[it][nLow:nHigh], color=mycolors[it], alpha = 0.8, label="Pred. KDE (iter %s)"%(it))
            plt.legend(loc=1, facecolor='white', edgecolor='white')

            # --- Text box info added on the top left part of the plot, with
            # - PV number
            # - number of tracks origiating from the PV
            # - PV position (in z and bin number)
#             textstr = '\n'.join((
#             r'Evt #%s' % (iEvt, ),     
#             r'$-$ PV #%s (of %s)' %(jj+1,nPlots ),
#             r'$-$ #tracks = %s' % (good_pvs[jj][2], ),     
#             r'$-$ PV position:',              
#             r'    z = $%.3f$ mm' % (good_pvs[jj][0]),             
#             r'    bin #%.0f' % (good_pv_bins[jj])))              
            #r'PV_bin = %.0f' % (good_pv_bins[jj], )))            
            props = dict(boxstyle='square', facecolor='white', edgecolor='white', alpha=0.5) #
            ax.text(0.05, 0.95, textstr, transform=ax.transAxes,verticalalignment='top', bbox=props,linespacing = 1.8)#, fontsize=14
            ax.set_xlabel("z (in bin #)")
            ax.set_ylabel("Arbitrary unit")
            fname = 'Evt%s_goodPV%s'%(iEvt,jj)
            #plt.savefig(str(output_path / f'{fname}.png'))
            plt.show()

In [ ]:
#quit()